# How to set up a host-guest system using SMIRNOFF99Frosst

In [1]:
import numpy as np
import subprocess as sp
import parmed as pmd
from openeye.oechem import *

from openforcefield.typing.engines.smirnoff import *
from openforcefield.utils import mergeStructure

David Mobley suggested:
> *However, I think I CAN offer one more piece of good news*: If it’s just a problem with the solvent (which I expect it is, at least in terms of the “too much space” issue) then you should be able to bypass it entirely by using what could be called a “mixed forcefield” system: Just parameterize your water as normal, and your host-guest part using SMIRNOFF, and join them up. (edited)

I'm going to combine some manual effort that went into `mixed-force-field.ipynb` and hints from David's notebook `mobley_testing.ipynb` to create a self-contained and (hopefully) clear notebook that will demonstrate how to build a system with the host and guest molecule parameterized with SMIRNOFF99Frosst combined with TIP3P water Joung-Cheatham ions.

# Create a PDB for the GAFF solvated system

In [2]:
def create_pdb_with_conect(solvated_pdb, amber_prmtop, output_pdb, path='./'):
    """
    Create a PDB file containing CONECT records.
    This is not very robust, please manually check the `cpptraj` output.
    `cpptraj` must be in your PATH.
    Parameters
    ----------
    solvated_pdb : str
        Existing solvated structure from e.g., Mobley's Benchmark Sets repository
    amber_prmtop : str
        AMBER (or other) parameters for the residues in the solvated PDB file
    output_pdb : str
        Output PDB file name
    path : str
        Directory for input and output files
    """
    cpptraj = \
    f'''
    parm {amber_prmtop}
    trajin {solvated_pdb}
    trajout {output_pdb} conect
    '''

    cpptraj_input = output_pdb + '.in'
    cpptraj_output = output_pdb + '.out'

    with open(path + cpptraj_input, 'w') as file:
        file.write(cpptraj)
    with open(path + cpptraj_output, 'w') as file:
        p = sp.Popen(['cpptraj', '-i', cpptraj_input], cwd=path,
                     stdout=file, stderr=file)
        output, error = p.communicate()
    if p.returncode == 0:
        print('PDB file written by cpptraj.')
    elif p.returncode == 1:
        print('Error returned by cpptraj.')
        print(f'Output: {output}')
        print(f'Error: {error}')
        p = sp.Popen(['cat', cpptraj_output], cwd=path, stdout=sp.PIPE)
        for line in p.stdout:
            print(line.decode("utf-8").strip(),)
    else:
        print(f'Output: {output}')
        print(f'Error: {error}')

In [3]:
def prune_conect(input_pdb, output_pdb, path='./'):
    """
    Delete CONECT records that correspond only to water molecules.
    This is necessary to be standards-compliant.
    This is not very robust.
    Parameters
    ----------
    input_pdb : str
        Input PDB file name
    output_pdb : str
        Output PDB file name
    path : str
        Directory for input and output files
    """
    p = sp.Popen(['grep', '-m 1', 'WAT', input_pdb], cwd=path, stdout=sp.PIPE)
    for line in p.stdout:
        first_water_residue = int(float(line.decode("utf-8").split()[1]))
        print(f'First water residue = {first_water_residue}')

    p = sp.Popen(['egrep', '-n', f'CONECT [ ]* {first_water_residue}', input_pdb],
                 cwd=path, stdout=sp.PIPE)
    for line in p.stdout:
        line_to_delete_from = int(float(line.decode("utf-8").split(':')[0]))
        print(f'Found first water CONECT entry at line = {line_to_delete_from}')

    with open(path + output_pdb, 'w') as file:
        sp.Popen(
         ['awk', f'NR < {line_to_delete_from}', input_pdb], cwd=path, stdout=file)

        sp.Popen(['echo', 'END'], cwd=path, stdout=file)

In [4]:
create_pdb_with_conect('original/solvated.inpcrd', 'original/solvated.prmtop', 'generated/solvated.pdb')
prune_conect('solvated.pdb', 'solvated_conect.pdb', path='generated/')

PDB file written by cpptraj.
First water residue = 175
Found first water CONECT entry at line = 9192


# Load the GAFF solvated PDB into ParmEd and split the system into its molecule components

In [5]:
topology = pmd.load_file('generated/solvated_conect.pdb')
components = topology.split()

Poking around with the `components` object, I can see that the host is listed first, then the guest, then the ions (Na and Cl, not sure of the ordering though), and then the water molecules.

In [6]:
topologies = pmd.Structure()
numbers = []
current_names = []

for c in components[0:2]:
    topologies += c[0]
    numbers.append(c[1])    
    current_names.append(c[0].residues[0].name)

In [7]:
print(topologies, numbers, current_names)

<Structure 161 atoms; 8 residues; 168 bonds; NOT parametrized> [{0}, {1}] ['MGO', 'BEN']


We now have a single `Structure` that contains the atoms and bonds from the host (MGO) and guest (BEN).

# Parameterize the host and guest with SMIRNOFF99Frosst using OpenEye

I'm going to start by loading the host and guest SYBYL-formatted `mol2` files into a list of `OEMol`s. I created these SYBYL-formatted files from files containing AM1-BCC charges and GAFF v1.7 Lennard-Jones and bonded parameters:

```
antechamber -i bcd.mol2 -fi mol2 -o bcd-sybyl.mol2 -fo mol2 -at sybyl
antechamber -i ben.mol2 -fi mol2 -o ben-sybyl.mol2 -fo mol2 -at sybyl -dr n 
# Disable `acdoctor` to handle carboxylate group
```

I think it should be possible to read in GAFF-formatted files directly, using a specific forcefield flavor, but I don't see the relvant flavor listed in their [documentation](https://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html#section-molreadwrite-flavoredinputandoutput). A search for "GAFF" comes up empty (except for one blog post). If I try to load the GAFF files without any special flavor, I get many things with atom type "Du," which I surmise to be a dummy atom type. I believe this can be remedied by running the files through `OETriposAtomNames`, but it is another thing that could go wrong, so for these reasons, I believe it is more straight-forward to simply start with standard atom names.

In [8]:
def load_mol2(filename, name=None, add_tripos=True):
    ifs = oemolistream()
    molecules = []
    if not ifs.open(filename):
        print(f'Unable to open {filename} for reading...')
    for mol in ifs.GetOEMols():
        if add_tripos:
            OETriposAtomNames(mol)
        if name:
            mol.SetTitle(name)
        # Add all the molecules in this file to a list, but only return the first one.
        molecules.append(OEMol(mol))
    return molecules[0]

In my testing, the atoms in beta-cyclodextrin need to be renamed (for example, using Tripos atom names), so every atom in the molecule has a unique name. Otherwise there is a problem matching the topology.

In [9]:
host = load_mol2('original/bcd-sybyl.mol2', name='MGO', add_tripos=True)
guest = load_mol2('original/ben-sybyl.mol2', name='BEN', add_tripos=False)
molecules = [host, guest]

In [10]:
def check_unique_atom_names(molecule):
    atoms = molecule.GetMaxAtomIdx()
    atom_names = set()
    for atom in range(atoms):
        atom_names.add(molecule.GetAtom(OEHasAtomIdx(atom)).GetName())
    print(f'{atoms} atoms in structure, {len(atom_names)} unique atom names.')
    assert atoms == len(atom_names)

In [11]:
check_unique_atom_names(host)
check_unique_atom_names(guest)

147 atoms in structure, 147 unique atom names.
14 atoms in structure, 14 unique atom names.


In [12]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(topologies.topology, molecules, 
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )

# Store the host-guest SMIRNOFF99Frosst parameters in a ParmEd `Structure`

In [13]:
host_guest_structure = pmd.openmm.topsystem.load_topology(topologies.topology, system, topologies.positions)

# Extract the water and ions from the GAFF solvated structure

In [14]:
def extract_water_and_ions(amber_prmtop, amber_inpcrd, host_residue, guest_residue, 
                           output_pdb, path='./'):
    """
    Create a PDB file containing just the water and ions.
    This is not very robust, please manually check the `cpptraj` output.
    `cpptraj` must be in your PATH.
    Parameters
    ----------
    amber_prmtop : str
        Existing solvated structure parameters from e.g., Mobley's Benchmark Sets repository
    amber_inpcrd : str
        Existing solvated structure coordinates
    host_residue : str
        Residue name of the host molecule (to be stripped)
    guest_residue : str
        Residue name of the guest molecule (to be stripped)
    output_pdb : str
        Output PDB file name
    path : str
        Directory for input and output files
    """
    
    cpptraj = \
    f'''
parm {amber_prmtop}
trajin {amber_inpcrd}
strip {host_residue}
strip {guest_residue}
trajout {output_pdb}
    '''
    cpptraj_input = output_pdb + '.in'
    cpptraj_output = output_pdb + '.out'

    with open(path + cpptraj_input, 'w') as file:
        file.write(cpptraj)
    with open(path + cpptraj_output, 'w') as file:
        p = sp.Popen(['cpptraj', '-i', cpptraj_input], cwd=path,
                     stdout=file, stderr=file)
        output, error = p.communicate()
    if p.returncode == 0:
        print('Water and ion PDB file written by cpptraj.')
    elif p.returncode == 1:
        print('Error returned by cpptraj.')
        print(f'Output: {output}')
        print(f'Error: {error}')
        p = sp.Popen(['cat', cpptraj_output], cwd=path, stdout=sp.PIPE)
        for line in p.stdout:
            print(line.decode("utf-8").strip(),)
    else:
        print(f'Output: {output}')
        print(f'Error: {error}')

In [22]:
def create_water_and_ions_parameters(input_pdb, output_prmtop, output_inpcrd, 
                                     water_model='tip3p', ion_model ='ionsjc_tip3p',
                                    path='./'):
    """
    Create AMBER coordinates and parameters for just the water and ions.
    This is not very robust, please manually check the `tleap` output.
    `tleap` must be in your PATH.
    Parameters
    ----------
    input_pdb : str
        PDB structure containing everything except the host and guest
    output_prmtop : str
        AMBER parameters for the water and ions
    output_inpcrd : str
        AMBER coordinates for the water and ions
    water_model : str
        Water model, must match AMBER `leaprc.water` and `frcmod`files
    ion_model : str
        Ion model, must match AMBER `leaprc.water` and `frcmod`files
    path : str
        Directory for input and output files
    """
    
    tleap = \
    f'''
source leaprc.protein.ff14sb
source leaprc.water.{water_model}
source leaprc.gaff
loadamberparams frcmod.{water_model}
loadamberparams frcmod.{ion_model}
mol = loadpdb {input_pdb}
saveamberparm mol {output_prmtop} {output_inpcrd}
quit
    '''
    tleap_input = output_prmtop + '.in'
    tleap_output = output_prmtop + '.out'

    with open(path + tleap_input, 'w') as file:
        file.write(tleap)
    with open(path + tleap_output, 'w') as file:
        p = sp.Popen(['tleap', '-f', tleap_input, '>', tleap_output], cwd=path,
                     stdout=file, stderr=file)
        output, error = p.communicate()
    if p.returncode == 0:
        print('Water and ion parameters and coordinates written by tleap.')
    elif p.returncode == 1:
        print('Error returned by tleap.')
        print(f'Output: {output}')
        print(f'Error: {error}')
        p = sp.Popen(['cat', tleap_output], cwd=path, stdout=sp.PIPE)
        for line in p.stdout:
            print(line.decode("utf-8").strip(),)
    else:
        print(f'Output: {output}')
        print(f'Error: {error}')

In [23]:
extract_water_and_ions('../original/solvated.prmtop', '../original/solvated.inpcrd', ':MGO', ':BEN', 
                           'water_ions.pdb', 'generated/')

Water and ion PDB file written by cpptraj.


In [24]:
create_water_and_ions_parameters('water_ions.pdb', 'water_ions.prmtop', 'water_ions.inpcrd', 
                                     water_model='tip3p', ion_model ='ionsjc_tip3p',
                                    path='generated/')

Water and ion parameters and coordinates written by tleap.


# Load in TIP3P and Joung-Cheatham ion parameters for the water and ions

In [18]:
water_and_ions = pmd.amber.AmberParm('generated/water_ions.prmtop', xyz='generated/water_ions.inpcrd')

# Merge the host-guest SMIRNOFF99Frosst parameters and the water and ion parameters to create a "mixed force field" ParmEd `Structure`

In [19]:
merged = mergeStructure(host_guest_structure, water_and_ions)

In [20]:
merged.save('generated/solvated_smirnoff.prmtop')

In [21]:
merged.save('generated/solvated_smirnoff.inpcrd')

Interesting, at this point, there are no box coordinates in the `inpcrd`, but I have been able to run a quick simulation using this `prmtop` and the original `inpcrd`. So that might be a good solution. Also, notably, when running this self-contained notebook with a fresh kernel, I do *not* get the `bond.type` error that I was experiencing before!